# NASA Mars News

In [27]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import os
import pandas as pd
import time
import requests
import pymongo

In [28]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


In [3]:
db = client.mars_news
collection = db.articles

In [4]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"


browser.visit(url)
html = browser.html

soup = bs(html, 'html.parser')
element = soup.select_one('ul.item_list li.slide')
title = element.find("div", class_="content_title").get_text()

In [6]:
#<div class="content_title" target="_self">/div>
#<div class="article_teaser_body"></div>
#<div class="list_text"> - parent???

news_title = element.find("div", class_="content_title").get_text()
news_paragraph = soup.find("div", class_="article_teaser_body").get_text()
print(f"News Title: {news_title}")
print(f"News Paragraph: {news_paragraph}")

News Title: Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover
News Paragraph: NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries. 


# JPL Mars Space Images - Featured Image

In [15]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [16]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [17]:
#<a class="button fancybox" data-description="xxxxxxx" FULL IMAGE

browser.click_link_by_id('full_image')

In [18]:
html = browser.html
image_soup = bs(html, "lxml")
img_url = image_soup.find('img', class_ = 'fancybox-image')['src']

In [19]:
#Make sure to save a complete url string for this image.

featured_img_url = "https://www.jpl.nasa.gov" + img_url
featured_img_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14417_ip.jpg'

# Mars Weather

In [25]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [32]:
url = "https://twitter.com/marswxreport?lang=en"

browser.visit(url)
html = browser.html

soup = bs(html, 'html.parser')

mars_weather = soup.find_all('div', class_='??????????????????')
mars_weather

AttributeError: 'NoneType' object has no attribute 'get_text'

In [ ]:
# <div aria-label="Timeline: Mars Weather's Tweets" class="css-1dbjc4n"> == $0  Top class with all tweets?
# <div class="css-1dbjc4n r-1ila09b r-qklmqi r-1adg3ll;"> Every tweet is named this so need the first tweet?
# I couldn't find the source of the tweet for this one :( I could see it but didn't know what was producing it.
# Also, I'm sure once I could find it, that I would need to create some kind of loop to grab just the first one?

# Mars Facts

In [12]:
url = "https://space-facts.com/mars/"
browser.visit(url)


In [13]:
facts = pd.read_html(url)
facts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [14]:
facts_df = facts[0]
facts_df.columns = ['Item', 'Value']
facts_df.head()

,Item,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [15]:
html_table = facts_df.to_html()
html_table.replace('\n', '') 
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Item</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>

# Mars Hemispheres

In [ ]:
# Cerberus - https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
# Schiaparelli - https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
# Syrtis Major - https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
# Valles Marineris - https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg

In [ ]:
valles_marineries_url = 
cerberus_url = 
schiaparelli_url = 
syrtis_major_url = 

In [ ]:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": valles_marineries_url},
    {"title": "Cerberus Hemisphere", "img_url": cerberus_url},
    {"title": "Schiaparelli Hemisphere", "img_url": schiaparelli_url},
    {"title": "Syrtis Major Hemisphere", "img_url": syrtis_major_url},
]
